# Logan Clark - College To NFL Trends (QB)

Predicting a players NFL Fantasy Football success by using College Football stats along with Random Forest Regression, and Neural Network Regression. 

**Data**

Evaluation Methodology: The current value metric I am using for each player is

Value = (QB1 Years/Years Played) + (Years Played/20)

My logic behind this is that it tries to mix how well a player performs by year along with their longevity.


Source: Pahowdy's Database

https://docs.google.com/spreadsheets/d/1D1D9rXo3BXX867qBju77VjZ9YIoAp-nXYFqSGDQgajU/edit#gid=224755041











In [1]:
!pip install tensorflow-addons

import math
import time

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
import pandas as pd

import tensorflow as tf
import tensorflow_addons as tfa

import torch

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics

tf.keras.backend.set_floatx('float64')

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py 

from helper_functions import create_tensorboard_callback, plot_loss_curves

     |████████████████████████████████| 1.1 MB 4.0 MB/s 
--2021-10-27 22:43:33--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-10-27 22:43:34 (76.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
def eval_model(model, X_test, y_test, hist):
    print("eval", model.evaluate(X_test, y_test))
    print("predict", model.predict(X_test[:10]))
    pd.DataFrame(hist.history).plot()
    plt.ylabel("loss")
    plt.xlabel("epochs")

metrics = [tf.metrics.mae, tfa.metrics.RSquare(dtype=tf.float32, y_shape=(1,))]
    

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Data Prep

In [31]:
data = pd.read_csv('/content/gdrive/My Drive/pahowdy_cfb_qb_training.csv')
rookie_data = pd.read_csv('/content/gdrive/My Drive/pahowdy_cfb_qb_rookies.csv')

features = (data.columns)[4:39]

print(features)

print(data.head())
print(rookie_data.head())

data = data.to_numpy()
rookie_data = rookie_data.to_numpy()
rookie_names = rookie_data[:,0]

#data = np.delete(data, 0, axis=0)
rookie_data = np.delete(rookie_data, 0, axis=0)

print(data.shape)
print(rookie_data.shape)



Index(['Conference', 'DR', 'DP', 'Last year designation', 'Age IN DRAFT YEAR',
       'Conference Drafted Rate', 'Conference Hit Rate',
       'Age Draft Hit Rate AVG', 'Rush ATT/Game', 'Pass Attempts',
       'Completions', 'Comp %', 'All Yards', 'Years played', 'G',
       'COMP% (Last Year)', 'Pass Yard Per Game (Last)', 'QB AVG YP(PassA)',
       'QB AVG AY/A',
       'CPOE* (*hacked versions - adjusted by PaYds/G instead of aDot- COMP% over expected using conference AVG since 2003)',
       'Comp AVG', 'AVG PassRat', 'AVG TANYA', 'BMI', 'weight', 'height', '40',
       '3Cone', 'shuttle', 'vert', 'broadInches', 'Burst Score', 'HaSS',
       'WaSS', '# of top 5  finishes'],
      dtype='object')
             Name            cfb_id  ... Years Played Longevity Score
0      Drew Brees      drew-brees-1  ...           20        1.750000
1   Philip Rivers   philip-rivers-1  ...           17        1.497059
2   Aaron Rodgers   aaron-rodgers-1  ...           16        1.487500
3  Russell 

In [32]:
def encode_cfb_data(input_data):
  ConferenceDict = {'ACC':0, 'ACC (Atlantic)':1, 'ACC (Coastal)':2, 'American':3, 'American (East)':4, 'American (West)':5,
                  'Big 12':6, 'Big 12 (North)':7, 'Big 12 (South)':8, 'Big East':9, 'Big Ten':10, 'Big Ten (East)':11,
                  'Big Ten (Leaders)':12, 'Big Ten (Legends)':13, 'Big Ten (West)':14, 'CUSA': 15, 'CUSA (East)':16,
                  'CUSA (West)':17, 'Ind':18, 'MAC (East)':19, 'MAC (West)':20, 'MWC':21, 'MWC (Mountain)':22,
                  'MWC (West)':23, 'Pac-10':24, 'Pac-12 (North)':25, 'Pac-12 (South)':26, 'SEC (East)':27,
                  'SEC (West)':28, 'Sun Belt':29, 'Sun Belt (East)':30, 'Sun Belt (West)':31, 'WAC':31, 'Big West':32}

  DraftRoundDict = {'UDFA':9}

  DraftPositionDict = {'UDFA':260}

  LastYearDesignationDict = {'SR':3, 'JR':2, 'SO':1, 'FR':0}

  for player in input_data:
    player[4] = ConferenceDict[(player[4].replace("\xa0", " "))]

    if player[5] in DraftRoundDict:
      player[5] = DraftRoundDict[player[5]]
    if player[6] in DraftPositionDict:
      player[6] = DraftPositionDict[player[6]]
    if player[7] in LastYearDesignationDict:
      player[7] = LastYearDesignationDict[player[7]]

  return input_data

In [33]:
# Data Encoding
data = encode_cfb_data(data)

rookie_data = encode_cfb_data(rookie_data)

In [34]:
# Split data into X and y

X = list()
y = list()

X_rookie = list()

for player in data:
  X.append(player[4:38])
  y.append(player[45])

for player in rookie_data:
  X_rookie.append(player[4:38])

print(len(X[0]))

X = np.array(X).astype(float) 
y = np.array(y).astype(float)

X_rookie = np.array(X_rookie).astype(float)

34


In [35]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Baseline Random Forest Model

In [36]:
rfReg = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)
rf_r_squared = rfReg.score(X_test, y_test)
rf_r_squared

0.45768404468823576

In [37]:
#List features by importance

feature_importance = rfReg.feature_importances_

feature_numbered_list = list()
for i in range(len(feature_importance)):
  feature_numbered_list.append((feature_importance[i], features[i]))


sorted_features = sorted(feature_numbered_list, reverse=True)
for i in sorted_features:
  print(i)

(0.39552029325425386, 'DP')
(0.05968939768436598, 'CPOE* (*hacked versions - adjusted by PaYds/G instead of aDot- COMP% over expected using conference AVG since 2003)')
(0.03877452608378554, 'Comp AVG')
(0.03316347605484448, 'Conference')
(0.03179536897437209, 'AVG PassRat')
(0.027584547079558076, 'Rush ATT/Game')
(0.026048216734329343, 'COMP% (Last Year)')
(0.022524245028837268, 'Pass Attempts')
(0.02069170671387713, 'Completions')
(0.01964003965275145, 'G')
(0.019254177296889656, 'broadInches')
(0.01776088671587807, 'height')
(0.0177432194893135, 'Pass Yard Per Game (Last)')
(0.017726808310875516, 'Conference Drafted Rate')
(0.017317463679372088, '3Cone')
(0.016796039855569253, 'BMI')
(0.01669247884304194, 'Comp %')
(0.0166149244769459, 'QB AVG YP(PassA)')
(0.016475471286767174, 'All Yards')
(0.016042984737373554, 'HaSS')
(0.016021254263704134, 'weight')
(0.015589045780046988, '40')
(0.015447848085779805, 'Conference Hit Rate')
(0.01525388123403195, 'QB AVG AY/A')
(0.0136711744852405

# Baseline Neural Network

In [26]:
tf.keras.backend.clear_session

<function keras.backend.clear_session>

In [38]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Pre-processing layer (normalization)
data_normalization = Sequential([
                                 preprocessing.Normalization()
], name="data_normalization")

# Setup input shape
input_shape = 34

# Assemble model
inputs = layers.Input(shape=input_shape, name="input_layer")
x = data_normalization(inputs)
x = layers.Dense(64)(x)
x = layers.LeakyReLU()(x)
x = layers.Dense(8)(x)
x = layers.LeakyReLU()(x)
outputs = layers.Dense(1, name="output_layer")(x)
model_0 = tf.keras.Model(inputs, outputs)

model_0.compile(loss=tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=metrics)

history_0 = model_0.fit(X_train, y_train, epochs=50, callbacks=[create_tensorboard_callback(dir_name="qb_trends",
                                                                                            experiment_name="model_0")])

Saving TensorBoard log files to: qb_trends/model_0/20211027-233453
Epoch 1/50
8/8 [==============================] - 1s 18ms/step - loss: 1737.4982 - mean_absolute_error: 1737.4983 - r_square: -17260604.0000
Epoch 2/50
8/8 [==============================] - 0s 2ms/step - loss: 299.0911 - mean_absolute_error: 299.0911 - r_square: -1154947.5000
Epoch 3/50
8/8 [==============================] - 0s 3ms/step - loss: 223.6423 - mean_absolute_error: 223.6423 - r_square: -636489.7500
Epoch 4/50
8/8 [==============================] - 0s 4ms/step - loss: 114.5674 - mean_absolute_error: 114.5674 - r_square: -166902.6094
Epoch 5/50
8/8 [==============================] - 0s 3ms/step - loss: 55.9645 - mean_absolute_error: 55.9645 - r_square: -36690.3359
Epoch 6/50
8/8 [==============================] - 0s 2ms/step - loss: 36.6734 - mean_absolute_error: 36.6734 - r_square: -18256.6230
Epoch 7/50
8/8 [==============================] - 0s 3ms/step - loss: 25.5558 - mean_absolute_error: 25.5558 - r_squa

In [39]:
model_0.evaluate(X_test, y_test)

2/2 [==============================] - 0s 7ms/step - loss: 8.0961 - mean_absolute_error: 8.0961 - r_square: -661.1632


[8.096095700417795, 8.096095858175504, -661.1632080078125]

In [40]:
model_1 = tf.keras.Sequential([
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(10),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(1),
])

model_1.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=metrics)

history = model_1.fit(X_train, y_train, epochs=50)

Epoch 1/50
8/8 [==============================] - 1s 4ms/step - loss: 164.6614 - mean_absolute_error: 164.6614 - r_square: -449456.9375
Epoch 2/50
8/8 [==============================] - 0s 4ms/step - loss: 22.8981 - mean_absolute_error: 22.8981 - r_square: -6745.3740
Epoch 3/50
8/8 [==============================] - 0s 4ms/step - loss: 11.1220 - mean_absolute_error: 11.1220 - r_square: -1543.0842
Epoch 4/50
8/8 [==============================] - 0s 4ms/step - loss: 7.9087 - mean_absolute_error: 7.9087 - r_square: -737.1850
Epoch 5/50
8/8 [==============================] - 0s 4ms/step - loss: 5.3262 - mean_absolute_error: 5.3262 - r_square: -345.2905
Epoch 6/50
8/8 [==============================] - 0s 4ms/step - loss: 2.8360 - mean_absolute_error: 2.8360 - r_square: -106.8774
Epoch 7/50
8/8 [==============================] - 0s 5ms/step - loss: 0.7842 - mean_absolute_error: 0.7842 - r_square: -7.8590
Epoch 8/50
8/8 [==============================] - 0s 4ms/step - loss: 0.5483 - mean_ab

In [41]:
model_1.evaluate(X_test, y_test)

2/2 [==============================] - 0s 9ms/step - loss: 0.2182 - mean_absolute_error: 0.2182 - r_square: 0.0929


[0.21817636201458593, 0.21817636461513498, 0.09291177988052368]

In [42]:
def normalized_predictions_with_names(predictions):
  pred_sum = sum(predictions)

  normalized_preds = [(float(i)/pred_sum)*1000 for i in predictions]

  names_preds = dict()

  for i in range(len(normalized_preds)):
      if type(normalized_preds[i]) is np.ndarray:
        names_preds[rookie_names[i]] = normalized_preds[i][0]
      else:
        names_preds[rookie_names[i]] = normalized_preds[i]   
          
  sorted_rookies = sorted(names_preds.items(), key=lambda x: x[1], reverse=True)

  # for i in sorted_rookies:
  #     print(i)
  return sorted_rookies

def sort_predictions_with_names(predictions):
  names_preds = dict()

  for i in range(len(predictions)):
      if type(predictions[i]) is np.ndarray:
        names_preds[rookie_names[i]] = predictions[i][0]
      else:
        names_preds[rookie_names[i]] = predictions[i]   
          
  sorted_rookies = sorted(names_preds.items(), key=lambda x: x[1], reverse=True)


  return sorted_rookies

In [46]:
rf_preds = rfReg.predict(X_rookie)
rf_preds_sorted = sort_predictions_with_names(rf_preds)
rf_preds_norm = normalized_predictions_with_names(rf_preds)

for i in rf_preds_norm:
  print(i)

('Justin Fields', 139.51678241899305)
('Sam Ehlinger', 119.02786372863744)
('Mac Jones', 118.37662606156402)
('Dustin Crum', 108.18883095081327)
('Hunter Johnson', 97.58138142683887)
('Zach Wilson', 77.26709821936194)
('Matt Corral', 67.76332564845717)
('Ian Book', 58.95193419828867)
("D'Eriq King", 39.80313159475517)
('McKenzie Milton', 38.857511713215324)
('Brock Purdy', 37.305770816978544)
('Josh Jackson', 27.099475027660247)
('Myles Brennan', 20.82852212869756)
('Jamie Newman', 16.573232661544292)
('Phil Jurkovec', 14.33360662620047)
('Davis Mills', 13.43775621206294)
('JT Daniels', 13.325774910295753)
('Jarrett Guarantano', 10.974167573184745)
('Dorian Thompson-Robinson', 10.078317159047206)
('Trevor Lawrence', 9.63039195197845)
('Sam Hartman', 2.7995325441797743)
('Michael Penix Jr.', -1.2743232503816919e-15)
('Quinten Dormady', -1.0078317159047214)
('Kellen Mond', -1.9676714454017796)
('K.J. Costello', -3.583401656550121)
('Kyle Trask', -4.031326863618885)
('Shane Buechele', -4.

In [45]:
print(rf_preds)

[-1.80000000e-02  1.77722222e-01  2.63222222e-01 -8.78571429e-03
 -3.35000000e-02  4.90000000e-02  5.31463119e-01  5.28555322e-01
  3.45000000e-01  4.83066500e-01  6.22946779e-01  4.30000000e-02
  1.66571429e-01 -1.60000000e-02 -4.10000000e-02 -2.45000000e-02
  1.73500000e-01  3.02565359e-01 -1.80000000e-02 -4.50000000e-03
  4.50000000e-02  7.40000000e-02  9.30000000e-02  6.40000000e-02
 -5.68989300e-18  1.25000000e-02  1.21000000e-01  4.35703907e-01
  6.00000000e-02  5.95000000e-02 -2.20000000e-02]


In [47]:
model_1_preds = model_1.predict(X_rookie)
model_1_preds_sorted = sort_predictions_with_names(model_1_preds)
model_1_preds_norm = normalized_predictions_with_names(model_1.predict(X_rookie))

for i in model_1_preds_norm:
  print(i)

('Justin Fields', 128.09775729568975)
('Hunter Johnson', 125.8469460246802)
('Dustin Crum', 123.97373332488165)
('Mac Jones', 119.20881888691319)
('Sam Ehlinger', 99.33831513738879)
('Matt Corral', 96.82660726246988)
('Ian Book', 87.72645222167233)
("D'Eriq King", 68.36616731232687)
('Josh Jackson', 44.342499612057516)
('Jarrett Guarantano', 24.825091626328003)
('Jamie Newman', 21.797995161600287)
('Dorian Thompson-Robinson', 18.94825085468178)
('Kyle Trask', 17.37495050651096)
('Shane Buechele', 17.01484061833685)
('Jake Bentley', 16.439455107284072)
('Davis Mills', 15.667623557627758)
('Michael Penix Jr.', 13.492740785130124)
('Sam Hartman', 13.05371366637733)
('Phil Jurkovec', 10.668376926265976)
('Quinten Dormady', 8.18723812546999)
('Trevor Lawrence', 4.983903517790531)
('Peyton Ramsey', 2.2494549080784934)
('Myles Brennan', -2.729962412572651)
('K.J. Costello', -5.585133622343173)
('Joe Milton', -5.633453925192495)
('JT Daniels', -6.204266767932294)
('McKenzie Milton', -7.5316567

# Training with Data that does not involve Draft Position to search for "Sleepers"

Theory: Players that's value increases the most when the data does not involve draft position are talented players who were not drafted high meaning they would have a higher % chance to breakout than the average player at their draft position.

In [48]:
# Split data into X and y, while removing features that are draft position or are related

X_sleeper = list()
y_sleeper = list()

X_sleeper_rookie = list()

for player in data:
  temp1 = player[7:14]
  temp2 = player[16:38]
  X_sleeper.append(list(temp1)+list(temp2))
  y_sleeper.append(player[45])

for player in rookie_data:
  temp1 = player[7:14]
  temp2 = player[16:38]
  X_sleeper_rookie.append(list(temp1)+list(temp2))

X_sleeper = np.array(X_sleeper).astype(float) 
y_sleeper = np.array(y_sleeper).astype(float)

X_sleeper_rookie = np.array(X_sleeper_rookie).astype(float)

In [50]:
# Split data into train and test
X_train_sleeper, X_test_sleeper, y_train_sleeper, y_test_sleeper = train_test_split(X_sleeper, y_sleeper, test_size=0.2, random_state=1)

X_train_sleeper = tf.convert_to_tensor(X_train_sleeper)
X_test_sleeper = tf.convert_to_tensor(X_test_sleeper)
y_train_sleeper = tf.convert_to_tensor(y_train_sleeper)
y_test_sleeper = tf.convert_to_tensor(y_test_sleeper)

In [51]:
#Train Random Forest
rf_sleeper = RandomForestRegressor(n_estimators=100).fit(X_train_sleeper, y_train_sleeper)
rf_r_squared_sleeper = rf_sleeper.score(X_test_sleeper, y_test_sleeper)
rf_r_squared_sleeper

0.4285893490533362

In [52]:
#List features by importance

sleeper_features = features[3:]

feature_importance = rf_sleeper.feature_importances_

feature_numbered_list = list()
for i in range(len(feature_importance)):
  feature_numbered_list.append((feature_importance[i], sleeper_features[i]))


sorted_features = sorted(feature_numbered_list, reverse=True)
for i in sorted_features:
  print(i)

(0.33625706825122614, 'Age Draft Hit Rate AVG')
(0.060782983039846784, 'QB AVG YP(PassA)')
(0.040495717170128634, 'CPOE* (*hacked versions - adjusted by PaYds/G instead of aDot- COMP% over expected using conference AVG since 2003)')
(0.038861135299921364, 'QB AVG AY/A')
(0.034002921328907636, 'Rush ATT/Game')
(0.031688322157220214, 'Pass Attempts')
(0.02999191130343821, 'Years played')
(0.02908233576429728, 'shuttle')
(0.027533080161873544, 'All Yards')
(0.024868904647952397, 'height')
(0.024501667103262893, 'Conference Hit Rate')
(0.024057773138671332, 'AVG PassRat')
(0.023004495596228202, 'broadInches')
(0.02266818411964537, 'COMP% (Last Year)')
(0.022400476171431993, 'AVG TANYA')
(0.022198185437382015, 'G')
(0.021312412217402592, 'Completions')
(0.02063274837348155, 'Comp AVG')
(0.020619092543960618, 'BMI')
(0.01986227303927991, 'Conference Drafted Rate')
(0.0197314979867839, 'Pass Yard Per Game (Last)')
(0.01963180154173018, 'Burst Score')
(0.017661045615590315, '3Cone')
(0.0174847

In [53]:
# Retrain model_1 on new data
model_1_sleeper = tf.keras.Sequential([
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(10),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(1),
])

model_1_sleeper.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=metrics)

history = model_1_sleeper.fit(X_train_sleeper, y_train_sleeper, epochs=50)

Epoch 1/50
8/8 [==============================] - 1s 4ms/step - loss: 184.0045 - mean_absolute_error: 184.0045 - r_square: -558865.4375
Epoch 2/50
8/8 [==============================] - 0s 4ms/step - loss: 22.3715 - mean_absolute_error: 22.3715 - r_square: -7689.1592
Epoch 3/50
8/8 [==============================] - 0s 4ms/step - loss: 4.0727 - mean_absolute_error: 4.0727 - r_square: -235.3665
Epoch 4/50
8/8 [==============================] - 0s 4ms/step - loss: 2.1546 - mean_absolute_error: 2.1546 - r_square: -81.6804
Epoch 5/50
8/8 [==============================] - 0s 4ms/step - loss: 6.2859 - mean_absolute_error: 6.2859 - r_square: -559.7510
Epoch 6/50
8/8 [==============================] - 0s 4ms/step - loss: 2.4180 - mean_absolute_error: 2.4180 - r_square: -84.2647
Epoch 7/50
8/8 [==============================] - 0s 5ms/step - loss: 1.1100 - mean_absolute_error: 1.1100 - r_square: -17.6770
Epoch 8/50
8/8 [==============================] - 0s 4ms/step - loss: 4.8555 - mean_absolu

In [54]:
model_1_sleeper.evaluate(X_test_sleeper, y_test_sleeper)

2/2 [==============================] - 0s 8ms/step - loss: 0.2938 - mean_absolute_error: 0.2938 - r_square: -0.1349


[0.29379950607976607, 0.29379952421796207, -0.13492166996002197]

In [55]:
rf_preds_sleeper = rf_sleeper.predict(X_sleeper_rookie)
rf_preds_norm_sleeper = normalized_predictions_with_names(rf_sleeper.predict(X_sleeper_rookie))

for i in rf_preds_norm_sleeper:
  print(i)


('Sam Ehlinger', 114.14573726349659)
('Mac Jones', 110.38367156693253)
('Dustin Crum', 95.8047497718702)
('Hunter Johnson', 74.91057588976518)
('Justin Fields', 70.7219949565754)
('Matt Corral', 59.37352952986408)
('Zach Wilson', 51.369781091929966)
('Ian Book', 51.31516468469269)
("D'Eriq King", 44.72365540684735)
('Brock Purdy', 39.716912550551825)
('Myles Brennan', 30.35642301778242)
('Jamie Newman', 28.614661041352274)
('Josh Jackson', 27.797099297076645)
('Davis Mills', 24.716431856008626)
('Phil Jurkovec', 24.29119442706048)
('McKenzie Milton', 24.135844530531898)
('JT Daniels', 21.73055418212839)
('Jarrett Guarantano', 20.891928045254975)
('Kyle Trask', 16.8462481113949)
('Dorian Thompson-Robinson', 16.743559195863202)
('Trevor Lawrence', 14.763506276407842)
('Kellen Mond', 9.787043486607457)
('Quinten Dormady', 6.8011658127272145)
('Jake Bentley', 6.13763744075383)
('Michael Penix Jr.', 4.5156791981153495)
('Sam Hartman', 4.182598487570567)
('K.J. Costello', 3.4097985789338203)

In [56]:
model_1_preds_sleeper = model_1_sleeper.predict(X_sleeper_rookie)
model_1_preds_norm_sleeper = normalized_predictions_with_names(model_1_sleeper.predict(X_sleeper_rookie))

for i in model_1_preds_norm_sleeper:
  print(i)

('Jarrett Guarantano', 62.6050363375735)
('Ian Book', 49.23214490365138)
('Justin Fields', 49.100724630457414)
("D'Eriq King", 47.97941537876302)
('Trevor Lawrence', 45.91607939807834)
('Joe Milton', 45.22161579296394)
('Jake Bentley', 45.1460268447209)
('Shane Buechele', 41.61575864440466)
('Kellen Mond', 39.810943883012484)
('Mac Jones', 38.13666052498806)
('Adrian Martinez', 36.85481006421652)
('Sam Ehlinger', 36.20566044007102)
('Matt Corral', 35.17060764142106)
('Zach Wilson', 33.635327612258365)
('K.J. Costello', 33.0067813833464)
('McKenzie Milton', 32.04670504617562)
('Dustin Crum', 32.023343490244784)
('Brock Purdy', 31.864883522263998)
('JT Daniels', 27.87231236972489)
('Dorian Thompson-Robinson', 26.305058306389686)
('Peyton Ramsey', 25.632851156780102)
('Quinten Dormady', 25.257906124203014)
('Michael Penix Jr.', 23.452132138062435)
('Sam Hartman', 21.77068232683093)
('Kyle Trask', 19.657750171510756)
('Davis Mills', 19.321436549616017)
('Hunter Johnson', 19.07289967435373)

In [57]:
# Create list based off differences

diff_list = list()

for i in range(len(rf_preds)):
  diff_list.append(model_1_preds_sleeper[i]-model_1_preds[i])

diff_dict = dict()

for i in range(len(diff_list)):
    if type(diff_list[i]) is np.ndarray:
      diff_dict[rookie_names[i]] = diff_list[i][0]
    else:
      diff_dict[rookie_names[i]] = diff_list[i]   
        
sorted_diff = sorted(diff_dict.items(), key=lambda x: x[1], reverse=True)

for i in sorted_diff:
  print(i)

('Jarrett Guarantano', 0.4368875934869182)
('Joe Milton', 0.36667958897236347)
('Trevor Lawrence', 0.34909924115396107)
('Kellen Mond', 0.3333692601112401)
('Jake Bentley', 0.31822185587738344)
('Adrian Martinez', 0.31135445938528195)
('Zach Wilson', 0.2985752071453764)
('Shane Buechele', 0.28930260759754145)
('Brock Purdy', 0.28109342715905317)
('K.J. Costello', 0.27083050829712546)
('McKenzie Milton', 0.26752612943590776)
('JT Daniels', 0.2319272738888509)
("D'Eriq King", 0.22782758575490236)
('Peyton Ramsey', 0.1960416178428976)
('Ian Book', 0.19566403629144277)
('Quinten Dormady', 0.1802265432557853)
('Dorian Thompson-Robinson', 0.1650991789159228)
('Michael Penix Jr.', 0.1545672172040997)
('Myles Brennan', 0.14278950177859423)
('Sam Hartman', 0.14233943737607802)
('Phil Jurkovec', 0.12344343487120576)
('Davis Mills', 0.11747308182041546)
('Kyle Trask', 0.1164068808115607)
('Justin Fields', 0.10708849223889588)
('Jamie Newman', 0.08809439339708051)
('Sam Ehlinger', 0.06837733485721